In [1]:
# Version Genie de Cosine del dia 09/06/2025

import cv2
from ultralytics import YOLO

# 1. Cargar el modelo YOLOv11 preentrenado
model = YOLO('yolo11n.pt')

# 2. Leer el video de entrada
path = 'C:/Users/gtoma/Master_AI_Aplicada/UEM_Master_AI_07042025/UEM_Trabajo/Sesiones_UEM/Manuel_Garcia_VISION/Actividad_3_YOLOV11/'
video_path = path + 'comma_small.mp4'
cap = cv2.VideoCapture(video_path)

# 3. Preparar el writer para el video de salida
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(path + 'video_con_detecciones.mp4', fourcc, int(cap.get(cv2.CAP_PROP_FPS)),
                    (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# 4. Diccionario para guardar trayectorias por ID
trajectories = dict()

# 5. Procesar frame a frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv11 inferencia con tracking activado
    results = model.track(frame, persist=True, tracker="botsort.yaml")  # Usa tracking basado en ByteTrack

    # Filtrar solo los vehículos y las personas (revisa clases: car, bus, truck, etc.)
    for det in results:
        if det.boxes is not None:
            for box in det.boxes:
                cls_id = int(box.cls[0])
                track_id = int(box.id[0]) if box.id is not None else None
                # Ajusta según las clases de vehículos en YOLOv11
                # 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench',
                # 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                # 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                # 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant',
                # 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 
                # 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
                if model.names[cls_id] in ['person', 'bycicle', 'car', 'motorcycle', 'bus', 'truck', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench']:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    # Dibujar bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                    cv2.putText(frame, f"{model.names[cls_id]} ID:{track_id}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

                    # Guardar centro del bounding box para la trayectoria
                    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
                    if track_id not in trajectories:
                        trajectories[track_id] = []
                    trajectories[track_id].append((cx, cy))

    # Dibujar trazas de cada objeto
    for track_id, points in trajectories.items():
        for i in range(1, len(points)):
            cv2.line(frame, points[i-1], points[i], (255,0,0), 2)

    # Escribir frame al video de salida
    out.write(frame)

# Liberar recursos
cap.release()
out.release()
print("¡Video generado con bounding boxes y trayectorias guardado como 'video_con_detecciones.mp4'!")


0: 384x640 1 traffic light, 78.9ms
Speed: 2.6ms preprocess, 78.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 47.7ms
Speed: 1.4ms preprocess, 47.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 52.9ms
Speed: 1.3ms preprocess, 52.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 44.8ms
Speed: 1.3ms preprocess, 44.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 43.9ms
Speed: 0.9ms preprocess, 43.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 44.5ms
Speed: 2.1ms preprocess, 44.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 43.2ms
Speed: 1.1ms preprocess, 43.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 42.6ms
Speed: 1.0ms preprocess, 42.6ms i

Notas importantes:
Cambia ['car', 'truck', 'bus', 'motorcycle'] según las clases que YOLOv11 tenga para vehículos (puedes imprimir model.names para verlas).
El tracking usa ByteTrack (incluido en Ultralytics). Puedes cambiar el tracker si lo deseas (ver docs Ultralytics).
Este código guarda el video con bbboxes y trayectorias en el mismo tamaño y fps que el original.

In [2]:
print(model.names)  # Imprime las clases del modelo YOLOv11

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [1]:
import ultralytics
print(ultralytics.__file__)

c:\Users\gtoma\anaconda3\envs\pytorch_13042025\Lib\site-packages\ultralytics\__init__.py
